In [2]:
import re

In [3]:
path = 'train_data_sila/train_data.csv'

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv(path)
data['full_text'] = data['Тема'] + ' ' + data['Описание']

In [6]:
data["Тема"] = data["Тема"].map(lambda x: re.sub(r'\s+', ' ', x.replace("//", "")).strip())
data["Описание"] = data["Описание"].map(lambda x: re.sub(r'\s+', ' ', x.replace("\n", " ").replace("_x000D_", "").strip()))
data["full_text"] = data["Тема"] + " " + data["Описание"]

In [7]:
data['Серийный номер'].unique()

array(['C223100360', 'C223094534', 'D251110041', 'C223014328',
       'C223014125', 'C223010123', 'C223091001', 'C223012430',
       'D252030021', 'c223012961', 'C223100312', 'C223090725',
       'C223102255', 'CKM01212505744', 'С111120102', 'D252110012',
       'С112040045', 'C222090950', 'C223090096', 'C223010035',
       'C223102298', 'C223091126', 'C223010076', 'C223013256',
       'С223090320', 'C223013523', 'C223014561', 'C223010345, С223092735',
       'D263120004', 'D114030017(FTKY614)', 'С111080061', 'C223010446',
       'D252030009',
       'C223011170, C223102198, C223011889, C223101201, C223100299, C223011531',
       'D251110015', 'C222090246', 'D254020003', 'C223091623',
       'C223012998 C223101389  C223102298   C223100876  C223100050',
       'D263090003', 'C223013250', 'C223011660', 'C233041042',
       'C223011088', 'С111080030', 'C223091624', 'D251110011',
       'C223014954', 'С223010395', 'C223012095', 'С223091324',
       'С223091153', 'C223090893', 'C223090123',

In [128]:
sample = data.sample(1)
print(sample['full_text'].values[0])
sample

СИЛА СХ2-4115-12; СИЛА CX2-4016-25; СИЛА СХ2-4066-36 обновить ПО до последней версии E244030001, E244030002, E244030003 Добрый день. Недавно приобретали у вас 3 схд, (СИЛА СХ2-4115-12; СИЛА CX2-4016-25; СИЛА СХ2-4066-36;) сейчас готовим оборудование к запуску и хотим обновить ПО до последней версии.


,index,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер,full_text
155,155,СИЛА СХ2-4115-12; СИЛА CX2-4016-25; СИЛА СХ2-4...,"Добрый день. Недавно приобретали у вас 3 схд, ...",СХД,Консультация,"E244030004, E244030002, E244030003",СИЛА СХ2-4115-12; СИЛА CX2-4016-25; СИЛА СХ2-4...


In [89]:
# 161

In [150]:
def to_latin(word):
    dict_ = {
        'А': 'A',
        'В': 'B',
        'С': 'C',
        'Е': 'E',
        'Р': 'P',
        'К': 'K',
        'Х': 'X',
        'О': 'O',
        'Н': 'H',
        'М': 'M',
        'Т': 'T',
    }
    new_word = ''
    for letter in word:
        if letter.isalpha() and not (ord('A') <= ord(letter) <= ord('z')):
            if letter in dict_:
                new_word += dict_[letter]
            else:
                new_word += letter
        else:
            new_word += letter
    return new_word

def ser_number(text):
    pattern = r'[A-Za-zА-Яа-я]*\d{6,}[A-Za-zА-Яа-я0-9]*'
    
    serial_numbers = re.findall(pattern, text)

    filtered_serial_numbers = [sn for sn in serial_numbers if not sn.isdigit() and len(sn) > 7]
    
    serial_numbers_array = np.array(filtered_serial_numbers)
    
    serial_numbers_array = [to_latin(x.upper()) for x in serial_numbers_array]
    
    return ', '.join(set(serial_numbers_array))

In [151]:
data['pred_ser'] = data['full_text'].apply(ser_number)

In [152]:
errors = data[data['Серийный номер'] != data['pred_ser']]

In [153]:
bad_idx = []

In [154]:
for idx, row in errors.iterrows():
    print(idx)
    print(row['Серийный номер'])
    print(row['pred_ser'])
    print(row['full_text'])
    print('-'*10)
    ans = input()
    if ans == 'y':
        bad_idx.append(idx)

3
C223014328

Повреждение матрицы Прошу произвести диагностику и сориентировать по условиям ремонта данного ноутбука.
----------


 y


4
C223014125
c223014125
sila HK2-1404 c223014125 asset 7101087 Добрый день! Прошу взять ноутбук sila HK2-1404 c223014125 asset 7101087 в гарантийный ремонт Не включается, нет индикации заряда.
----------


 y


6
C223091001
C223091001, C223091126
НК2-3404 Мерцание экрана C223091001 У нас же третий из 50 ноутбуков СИЛА проявляет слабость с экраном, вероятно, дело в шлейфе. Грустно то, что все 3 этих ноутбука не использовались «в полях» а стояли в офисе, их почти не открывали – закрывали. Известная проблема? Мерцает экран, пока есть 2 серийника, 1 ноутбук в пути: C223091001 C223091126 Возможен ли ремонт не в Москве\Области? У нас есть офис в Самаре и в Санкт-Петербурге.
----------


...

 y


192
C222091750

Отказ блока питания Добрый день! У нас перестал работать зарядник от ноутбука НК2-1404 (JHD-AD065B-BA-PD05) Можете предложить решение по ремонту/замене?
----------


 n


193
D263080001

Обновление BIOS и BMC Cube Добрый день! Если были обновления этих компонентов, прошу предоставить для скачивания. И появился ли у вас какой-то портал с прошивками и инструкциями по оборудованию Сила?
----------


 n


In [159]:
data.iloc[bad_idx]

,index,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер,full_text,pred_ser
3,3,Повреждение матрицы,Прошу произвести диагностику и сориентировать ...,Ноутбук,Матрица,C223014328,Повреждение матрицы Прошу произвести диагности...,
4,4,sila HK2-1404 c223014125 asset 7101087,Добрый день! Прошу взять ноутбук sila HK2-1404...,Ноутбук,Материнская плата,C223014125,sila HK2-1404 c223014125 asset 7101087 Добрый ...,c223014125
9,9,Звук и микрофон не активны на ноутбуке Сила ru...,перестали работать Звук и микрофон не активны ...,Ноутбук,Материнская плата,c223012961,Звук и микрофон не активны на ноутбуке Сила ru...,ruc223012961
40,40,заявка на гарантийный ремонт\замену,Добрый день. Серийный номер: С223013250 Неиспр...,Ноутбук,Блок питания,C223013250,заявка на гарантийный ремонт\замену Добрый ден...,С223013250
63,63,СИЛА CX-1015-12 HDD fault s/n E213120001. Выше...,"Вышел из строя диск, пришедший на замену по га...",СХД,Диск,Е213120001,СИЛА CX-1015-12 HDD fault s/n E213120001. Выше...,E213120001
164,164,СИЛА НК2-1404 v2 Неисправность ноутбука С22301...,"Прошу принять в гарантийный ремонт ноутбук ""СИ...",Ноутбук,Диск,C223011138,СИЛА НК2-1404 v2 Неисправность ноутбука С22301...,С223011138
169,169,Ремонт,"Добрый день, У меня проблема с ноутбуком «Сила...",Ноутбук,Материнская плата,C223091266,"Ремонт Добрый день, У меня проблема с ноутбуко...",С223091266
188,188,Ноутбук СИЛА НК2-3404 рябь экрана c223091025,"Коллеги, привет! Столкнулись с проблемой на но...",Ноутбук,Матрица,C223091025,Ноутбук СИЛА НК2-3404 рябь экрана c223091025 К...,c223091025
191,191,Ноутбук НК2-1404 Не работает клавиша F1 или ср...,"Добрый день, после ремонта ноутбука перестала ...",Ноутбук,Клавиатура,С223010731,Ноутбук НК2-1404 Не работает клавиша F1 или ср...,задержкойС223010731


In [155]:
len(errors)

77

In [156]:
len(bad_idx)

9

In [ ]:
# CKM00193901783

In [133]:
data['pred_ser'].astype(int).sum()

0

In [130]:
ser_number(sample['full_text'].values[0])

'E244030001, E244030002, E244030003'

In [178]:
import re
import numpy as np

# Пример текста, содержащего серийные номера
text = sample['full_text'].values[0]
# Регулярное выражение для поиска серийных номеров
# pattern = r'[A-Za-zА-Яа-я]*\d{6,}[A-Za-zА-Яа-я0-9]*'
pattern = r'[A-Za-zА-Яа-я]*\d{6,}[A-Za-zА-Яа-я0-9]*'

# Извлечение серийных номеров
serial_numbers = re.findall(pattern, text)

# Фильтрация серийных номеров, чтобы исключить отдельно стоящие цифры
filtered_serial_numbers = [sn for sn in serial_numbers if not sn.isdigit() and len(sn) > 7]

# Преобразование в массив (если нужно)
serial_numbers_array = np.array(filtered_serial_numbers)

serial_numbers_array = [to_latin(x.upper()) for x in serial_numbers_array]

print(set(serial_numbers_array))

{'E244030001', 'E244030003', 'E244030002'}


In [67]:
sample['full_text'].values[0]

'Неисправность сервера СИЛА 46900 Коллеги, добрый день. Просьба принять запрос. Неисправность сервера СИЛА // 46900 Модель: Сервер Сила S/N D252030050 Описание: Все ошибки которые удалось получить приведены ниже , далее модуль управления перестал отвечать и в данный момент не доступен. Service phosphor-ipmi-host.service has exited unsuccessfully. ila-rwfs-watchdog[20328]: /dev/mtdblock5 is full: coredumps and rotated logs were saved: /tmp/coredump-rwfs.tar.xz. Core_(all core)_CPU2 sensor crossed a critical high threshold going high. Reading=75.000000 Threshold=75.000000 .'

In [161]:
!pip install unidecode


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [165]:
from unidecode import unidecode

# Текст на кириллице
cyrillic_text = "С123123"

# Преобразование на латиницу
latin_text = unidecode(cyrillic_text)

print(latin_text)  # Вывод: Privet, mir!

S123123


In [175]:
def to_latin(word):
    dict_ = {
        'А': 'A',
        'В': 'B',
        'С': 'C',
        'Е': 'E',
        'Р': 'P',
        'К': 'K',
        'Х': 'X',
        'О': 'O',
        'Н': 'H',
        'М': 'M',
        'Т': 'T',
    }
    new_word = ''
    for letter in word:
        if letter.isalpha() and not (ord('A') <= ord(letter) <= ord('z')):
            if letter in dict_:
                new_word += dict_[letter]
            else:
                new_word += letter
        else:
            new_word += letter
    return new_word

In [177]:
to_latin('АВС')

'ABC'

In [ ]:
A B C E P K X O H M T

In [170]:
ord('Z')

90